In [ ]:
# Импортируем необходимые библиотеки
import os
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import cv2
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import nltk
from collections import Counter
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torch.cuda.amp import GradScaler, autocast

In [ ]:
# Установим случайные начальные значения для воспроизводимости
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
# Проверим доступность GPU видеокарты
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")

In [ ]:
# 1. Предварительная обработка данных

class VideoDataset(Dataset):
    """Датасет для работы с видео и их описаниями"""
    
    def __init__(self, feature_dir, caption_file, vocab, max_frames=40):
        """Инициализация датасета
        
        Аргументы:
            feature_dir (str): Путь к директории с предвычисленными признаками
            caption_file (str): Файл с описаниями в формате "video_id описание"
            vocab (Vocabulary): Объект словаря для токенизации
            max_frames (int): Макс. количество кадров на видео
        """
        self.feature_dir = feature_dir
        self.max_frames = max_frames
        self.vocab = vocab
        
        # Загрузка и парсинг описаний
        self.captions = self._load_captions(caption_file)
        self.video_ids = list(self.captions.keys())
    
    def _load_captions(self, caption_file):
        """Загружает описания из файла"""
        captions = {}
        with open(caption_file, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split(' ', 1)
                if len(parts) == 2:
                    video_id, caption = parts
                    captions.setdefault(video_id, []).append(caption)
        return captions
    
    def __len__(self):
        return len(self.video_ids)
    
    def __getitem__(self, idx):
        """Получает один элемент датасета по индексу"""
        video_id = self.video_ids[idx]
        
        # Загрузка предвычисленных признаков
        features = self._load_features(video_id)
        
        # Выбор случайного описания и токенизация
        caption = self._process_caption(video_id)
        
        return features, caption
    
    def _load_features(self, video_id):
        """Загружает признаки видео из файла"""
        feature_path = os.path.join(self.feature_dir, f"{video_id}.npy")
        features = np.load(feature_path)
        
        # Проверяем и корректируем размерность
        if features.ndim > 2:            
            features = features.reshape(features.shape[0], -1) # Преобразуем к [seq_len, feature_dim]
        return torch.FloatTensor(features).to(device)
    
    def _process_caption(self, video_id):
        """Токенизирует и преобразует описание в тензор"""
        caption = np.random.choice(self.captions[video_id])
        tokens = nltk.tokenize.word_tokenize(caption.lower())
        caption = [self.vocab('<start>')] + [self.vocab(token) for token in tokens] + [self.vocab('<end>')]
        return torch.LongTensor(caption)

class Vocabulary:
    """Словарь для преобразования слов в индексы"""
    
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0
        self._add_special_tokens()
    
    def _add_special_tokens(self):
        """Добавляет специальные токены"""
        for token in ['<pad>', '<start>', '<end>', '<unk>']:
            self.add_word(token)
    
    def add_word(self, word):
        """Добавляет слово в словарь"""
        if word not in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
    
    def __call__(self, word):
        """Возвращает индекс слова или токен <unk>"""
        return self.word2idx.get(word, self.word2idx['<unk>'])
    
    def __len__(self):
        return len(self.word2idx)

def build_vocab(caption_file, threshold=3):
    """Строит словарь на основе файла с описаниями"""
    counter = Counter()
    
    # Подсчет частот слов
    with open(caption_file, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split(' ', 1)
            if len(parts) == 2:
                counter.update(nltk.tokenize.word_tokenize(parts[1].lower()))
    
    # Фильтрация по порогу
    vocab = Vocabulary()
    for word, count in counter.items():
        if count >= threshold:
            vocab.add_word(word)
    
    return vocab

def precompute_features(video_dir, output_dir, batch_size=16):
    """Предварительно вычисляет признаки видео с помощью ResNet152"""
    
    os.makedirs(output_dir, exist_ok=True)
    
    # Инициализация модели для извлечения признаков
    model = models.resnet152(weights=models.ResNet152_Weights.IMAGENET1K_V1)
    feature_extractor = nn.Sequential(*list(model.children())[:-1]).to(device)
    feature_extractor.eval()
    
    # Трансформации для кадров
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])
    
    # Обработка видеофайлов
    video_files = [f for f in os.listdir(video_dir) if f.endswith(('.mp4', '.avi'))]
    
    for i in range(0, len(video_files), batch_size):
        batch_files = video_files[i:i + batch_size]
        
        for video_file in tqdm(batch_files, desc=f"Batch {i//batch_size + 1}"):
            
            video_id = os.path.splitext(video_file)[0]
            feature_path = os.path.join(output_dir, f"{video_id}.npy") # Проверяем, существуют ли уже признаки
            if os.path.exists(feature_path):
                continue  # Признаки уже есть, пропускаем

            video_path = os.path.join(video_dir, video_file)
            # Извлечение кадров
            frames = extract_frames(video_path)
            
            # Извлечение признаков
            features = []
            with torch.no_grad():
                for frame in frames:
                    if frame.ndim == 3:  # Проверка валидности кадра
                        frame = transform(frame).unsqueeze(0).to(device)
                        feature = feature_extractor(frame).squeeze().cpu().numpy()
                        features.append(feature)
            
            # Сохранение признаков
            features = np.stack(features, axis=0)
            np.save(os.path.join(output_dir, f"{video_id}.npy"), features)

def extract_frames(video_path, max_frames=40):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Could not open video: {video_path}")
    
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    sample_rate = max(1, frame_count // max_frames)
    frames = []
    
    for i in range(0, frame_count, sample_rate):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        if len(frames) >= max_frames:
            break
    
    cap.release()
    
    # Если кадров меньше, чем нужно, дублируем последний кадр вместо нулей
    if len(frames) < max_frames and len(frames) > 0:
        last_frame = frames[-1]
        frames.extend([last_frame for _ in range(max_frames - len(frames))])
    elif len(frames) == 0:  # Если кадров нет вообще
        frames = [np.zeros((224, 224, 3), dtype=np.uint8) for _ in range(max_frames)]
    
    return frames

In [ ]:
# 2. Извлечение признаков

class FeatureExtractor:
    """
    Класс для извлечения признаков из видеокадров с использованием предобученной CNN
    Основные функции:
    - Инициализация предобученной модели CNN (по умолчанию ResNet152)
    - Преобразование входных кадров к нужному формату
    - Извлечение признаков из каждого кадра
    """
    def __init__(self, cnn_model=None):
        if cnn_model is None:
            # Загрузка предобученной ResNet152
            cnn_model = models.resnet152(weights=models.ResNet152_Weights.IMAGENET1K_V1)
            # Удаляем последний классификационный слой
            self.model = nn.Sequential(*list(cnn_model.children())[:-1])
        else:
            self.model = cnn_model

        # Перенос модели на устройство (GPU/CPU) и перевод в режим оценки
        self.model = self.model.to(device)
        self.model.eval()
        
        # Определение преобразований для входных изображений
        self.transform = transforms.Compose([
            transforms.ToPILImage(),               # Конвертация в PIL Image
            transforms.Resize((224, 224)),         # Изменение размера под вход сети
            transforms.ToTensor(),                 # Конвертация в тензор
            transforms.Normalize(                  # Нормализация
                mean=[0.485, 0.456, 0.406],        # Средние значения ImageNet
                std=[0.229, 0.224, 0.225]          # Стандартные отклонения ImageNet
            )
        ])
    
    def extract_features(self, frames):
        """
        Извлекает признаки из списка кадров
        
        Аргументы:
            frames (list): Список кадров в формате numpy arrays
            
        Возвращает:
            torch.Tensor: Извлеченные признаки размерности [число_кадров, размерность_признака]
            
        Процесс работы:
        1. Применение преобразований к каждому кадру
        2. Извлечение признаков с помощью CNN
        3. Накопление и объединение признаков
        """
        features = []
        
        with torch.no_grad(): # Отключаем вычисление градиентов для ускорения
            for frame in frames:
                # Применяем преобразования и добавляем batch-размерность
                frame = self.transform(frame).unsqueeze(0).to(device)
                
                # Извлекаем признаки
                feature = self.model(frame)
                feature = feature.squeeze() # Удаляем лишние размерности
                
                features.append(feature.cpu()) # Переносим на CPU для экономии памяти
        
        return torch.stack(features) # Объединяем все признаки в один тензор

In [ ]:
# 3. Архитектура модели

class Encoder(nn.Module):
    """
    Видео-энкодер для обработки признаков кадров с учетом временной информации
    Использует двунаправленный LSTM для анализа последовательности кадров
    
    Основные функции:
    - Обработка признаков отдельных кадров
    - Учет временных зависимостей между кадрами
    - Подготовка скрытых состояний для декодера
    """
    def __init__(self, feature_dim, hidden_dim, num_layers=1, dropout=0.5):
        """
        Инициализация энкодера
        
        Аргументы:
            feature_dim (int): Размерность входных признаков кадра
            hidden_dim (int): Размерность скрытого слоя LSTM
            num_layers (int): Количество слоев LSTM
            dropout (float): Вероятность дропаута
        """
        super(Encoder, self).__init__()
        
        self.feature_dim = feature_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # Двунаправленный LSTM для временного кодирования
        self.lstm = nn.LSTM(
            input_size=feature_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,          # Первая размерность - batch
            bidirectional=True,       # Двунаправленная архитектура
            dropout=dropout if num_layers > 1 else 0  # Дропаут только для многослойных LSTM
        )
        
    def forward(self, features):
        """
        Прямой проход через энкодер
        
        Аргументы:
            features (torch.Tensor): Признаки видеокадров размерности 
                                    [batch_size, seq_len, feature_dim]
            
        Возвращает:
            tuple: (outputs, hidden)
                - outputs: Выходы LSTM [batch_size, seq_len, hidden_dim*2]
                - hidden: Кортеж (скрытое состояние, состояние ячейки)
        """
         # Проверяем размерность входных данных
        if features.dim() > 3:
            batch_size, seq_len = features.size(0), features.size(1)
            # Преобразуем к [batch_size, seq_len, feature_dim]
            features = features.view(batch_size, seq_len, -1)
            
        # Убеждаемся, что последнее измерение имеет правильный размер
        if features.size(-1) != self.feature_dim:
            raise ValueError(f"Неверная размерность признаков: ожидается {self.feature_dim}, получено {features.size(-1)}")
        
        # Пропускаем признаки через LSTM
        outputs, hidden = self.lstm(features)
        
        return outputs, hidden


class AttentionLayer(nn.Module):
    """
    Слой внимания для выделения наиболее релевантных частей видео
    Реализует механизм внимания на основе скалярного произведения
    
    Основные функции:
    - Вычисление весов внимания для каждого кадра
    - Создание контекстного вектора
    """
    def __init__(self, encoder_dim, decoder_dim):
        """
        Инициализация слоя внимания
        
        Аргументы:
            encoder_dim (int): Размерность выхода энкодера
            decoder_dim (int): Размерность скрытого состояния декодера
        """
        super(AttentionLayer, self).__init__()

        # Линейные преобразования для вычисления внимания        
        self.encoder_attn = nn.Linear(encoder_dim, decoder_dim)
        self.full_attn = nn.Linear(decoder_dim, 1)
        
    def forward(self, encoder_outputs, decoder_hidden):
        """
        Прямой проход через слой внимания
        
        Аргументы:
            encoder_outputs (torch.Tensor): Выходы энкодера [batch_size, seq_len, encoder_dim]
            decoder_hidden (torch.Tensor): Скрытое состояние декодера [batch_size, decoder_dim]
            
        Возвращает:
            tuple: (context, attention_weights)
                - context: Контекстный вектор [batch_size, encoder_dim]
                - attention_weights: Веса внимания [batch_size, seq_len]
        """
        # Проецируем выходы энкодера в пространство декодера
        # Размерность: [batch_size, seq_len, decoder_dim]
        attn_proj = self.encoder_attn(encoder_outputs)
        
        # Добавляем размерность для совместимости
        # Размерность: [batch_size, 1, decoder_dim]
        decoder_hidden = decoder_hidden.unsqueeze(1)
        
        # Вычисляем оценки внимания через тангенс
        # Размерность: [batch_size, seq_len, 1]
        attn_scores = self.full_attn(torch.tanh(attn_proj + decoder_hidden))
        
        # Нормализуем оценки в веса с помощью softmax
        # Размерность: [batch_size, seq_len]
        attn_weights = F.softmax(attn_scores.squeeze(2), dim=1)
        
        # Вычисляем взвешенную сумму выходов энкодера
        # Размерность: [batch_size, encoder_dim]
        context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs).squeeze(1)
        
        return context, attn_weights


class Decoder(nn.Module):
    """
    Декодер подписей, генерирующий описание слово за словом
    Использует механизм внимания для фокусировки на релевантных частях видео
    
    Основные компоненты:
    - Слой эмбеддинга слов
    - Механизм внимания
    - LSTM ячейка
    - Выходной слой
    """
    def __init__(self, vocab_size, embed_dim, encoder_dim, hidden_dim, attention_dim, dropout=0.5):
        """
        Инициализация декодера
        
        Аргументы:
            vocab_size (int): Размер словаря
            embed_dim (int): Размерность эмбеддинга слов
            encoder_dim (int): Размерность выхода энкодера
            hidden_dim (int): Размерность скрытого слоя LSTM
            attention_dim (int): Размерность механизма внимания
            dropout (float): Вероятность дропаута
        """
        super(Decoder, self).__init__()
        
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.encoder_dim = encoder_dim
        self.hidden_dim = hidden_dim
        
        # Слой эмбеддинга слов
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # Слой внимания
        self.attention = AttentionLayer(encoder_dim, hidden_dim)
        
        # LSTM ячейка
        self.lstm = nn.LSTMCell(embed_dim + encoder_dim, hidden_dim)
        
        # Слой дропаута
        self.dropout = nn.Dropout(dropout)
        
        # Выходной полносвязный слой
        self.fc = nn.Linear(hidden_dim, vocab_size)
        
        # Инициализация весов
        self.init_weights()

    def init_weights(self):
        """Инициализация весов для эмбеддингов и выходного слоя."""
        self.embedding.weight.data.uniform_(-0.1, 0.1)  # Равномерная инициализация
        self.fc.bias.data.fill_(0)                      # Нулевые смещения
        self.fc.weight.data.uniform_(-0.1, 0.1)         # Равномерная инициализация
        
    def init_hidden_state(self, batch_size):
        """
        Инициализация скрытого состояния LSTM
        
        Аргументы:
            batch_size (int): Размер батча
            
        Возвращает:
            tuple: (скрытое состояние, состояние ячейки)
        """
        h = torch.zeros(batch_size, self.hidden_dim).to(device)
        c = torch.zeros(batch_size, self.hidden_dim).to(device)
        return h, c
        
    def forward(self, encoder_outputs, captions, lengths):
        """
        Прямой проход через декодер (обучение)
        
        Аргументы:
            encoder_outputs (torch.Tensor): Выходы энкодера [batch_size, seq_len, encoder_dim]
            captions (torch.Tensor): Истинные подписи [batch_size, max_caption_length]
            lengths (list): Фактические длины подписей
            
        Возвращает:
            torch.Tensor: Предсказания [batch_size, max_caption_length, vocab_size]
        """
        batch_size = encoder_outputs.size(0)
        
        # Сортируем данные по убыванию длины (для эффективности)
        lengths, sort_idx = torch.sort(lengths, descending=True)
        encoder_outputs = encoder_outputs[sort_idx]
        captions = captions[sort_idx]
        
        # Инициализируем состояние LSTM
        h, c = self.init_hidden_state(batch_size)
        
        # Определяем максимальную длину в батче
        max_length = lengths[0].item()
        
        # Инициализируем тензор предсказаний
        predictions = torch.zeros(batch_size, max_length, self.vocab_size).to(device)
        
        # Получаем эмбеддинги слов
        embeddings = self.embedding(captions)
        
        # Инициализируем контекстный вектор
        context, _ = self.attention(encoder_outputs, h)
        
        # Генерируем слова последовательно
        for t in range(max_length):
            # Объединяем эмбеддинг и контекстный вектор
            lstm_input = torch.cat([embeddings[:, t], context], dim=1)
            
            # Прямой проход через LSTM
            h, c = self.lstm(lstm_input, (h, c))
            
            # Вычисляем новый контекст
            context, _ = self.attention(encoder_outputs, h)
            
            # Генерируем предсказание следующего слова
            output = self.fc(self.dropout(h))
            predictions[:, t] = output
        
        return predictions
    
    # алгоритм лучевого поиска (beam search) для генерации последовательности
    def beam_search(self, encoder_outputs, vocab, max_length=20, beam_size=5):
        # Получаем размер батча из выходных данных энкодера
        batch_size = encoder_outputs.size(0)
        # Определяем устройство (CPU/GPU) для создания новых тензоров
        device = encoder_outputs.device
    
        # Инициализация скрытых состояний LSTM декодера
        h, c = self.init_hidden_state(batch_size)
        
        # Получаем индексы служебных токенов
        start_idx = vocab.word2idx['<start>']  # Токен начала последовательности
        end_idx = vocab.word2idx['<end>']      # Токен конца последовательности
    
        # Инициализируем список "лучей" (кандидатов)
        # Каждый луч: (суммарный логарифм вероятности, последовательность, h, c)
        beams = [(0.0, [start_idx], h, c)]
        completed = []  # Завершенные последовательности
    
        # Основной цикл генерации по шагам
        for _step in range(max_length):
            all_candidates = []
            
            # Обрабатываем каждый текущий луч
            for score, seq, h, c in beams:
                # Проверка завершения последовательности
                if seq[-1] == end_idx:
                    completed.append((score, seq))
                    continue  # Пропускаем завершенные последовательности
    
                # Подготовка последнего токена
                last = torch.LongTensor([seq[-1]]).to(device)
                
                # Forward pass через модель
                emb = self.embedding(last)          # Получаем эмбеддинг токена [1, E]
                context, _ = self.attention(encoder_outputs, h)  # Контекст внимания
                lstm_input = torch.cat([emb, context], dim=1)     # Объединение входов
                h_new, c_new = self.lstm(lstm_input, (h, c))      # Обновление состояний
                out = self.fc(h_new)                # Преобразование в пространство слов
                logp = F.log_softmax(out, dim=1)    # Логарифмические вероятности [1, V]
    
                # Выбор топ-k кандидатов
                topk_logp, topk_idx = logp.topk(beam_size, dim=1)
                
                # Генерация новых кандидатов для текущего луча
                for k in range(beam_size):
                    next_token = topk_idx[0, k].item()    # Индекс следующего токена
                    next_score = topk_logp[0, k].item()   # Логарифм вероятности
                    
                    # Обновляем параметры для нового кандидата:
                    new_score = score + next_score        # Накопленный счет
                    new_seq = seq + [next_token]         # Расширенная последовательность
                    all_candidates.append((new_score, new_seq, h_new.clone(), c_new.clone()))
    
            # Выходим, если нет кандидатов для продолжения
            if not all_candidates:
                break
    
            # Сортируем всех кандидатов по убыванию суммарного счета
            all_candidates.sort(key=lambda x: x[0], reverse=True)
            # Отбираем топ-beam_size кандидатов
            beams = all_candidates[:beam_size]
    
            # Проверка раннего завершения (все лучи завершены)
            if all(b[1][-1] == end_idx for b in beams):
                completed.extend(beams)
                break
    
        # Обработка случая, когда ни один луч не был завершен
        if not completed:
            completed = beams
    
        # Выбор лучшей последовательности
        completed.sort(key=lambda x: x[0], reverse=True)
        best_seq = completed[0][1]
    
        # Постобработка последовательности:
        # 1. Удаляем стартовый токен
        if best_seq and best_seq[0] == start_idx:
            best_seq = best_seq[1:]
        
        # 2. Обрезаем после первого end токена (не включая его)
        if end_idx in best_seq:
            cut = best_seq.index(end_idx)
            best_seq = best_seq[:cut]
        
        # 3. Удаляем любые оставшиеся end токены (защита)
        best_seq = [tok for tok in best_seq if tok != end_idx]
    
        # Возвращаем последовательность как тензор
        return torch.LongTensor(best_seq).to(device)


class VideoCaptioningModel(nn.Module):
    """
    Полная модель генерации подписей к видео, объединяющая:
    - Энкодер видео
    - Декодер с механизмом внимания
    """
    def __init__(self, vocab_size, feature_dim=2048, embed_dim=512, encoder_dim=512, 
                 decoder_dim=512, attention_dim=512, dropout=0.5):
        """
        Инициализация модели
        
        Аргументы:
            vocab_size (int): Размер словаря
            feature_dim (int): Размерность входных признаков
            embed_dim (int): Размерность эмбеддингов слов
            encoder_dim (int): Размерность скрытого слоя энкодера
            decoder_dim (int): Размерность скрытого слоя декодера
            attention_dim (int): Размерность механизма внимания
            dropout (float): Вероятность дропаута
        """
        super(VideoCaptioningModel, self).__init__()
        
        # Инициализация энкодера
        self.encoder = Encoder(
            feature_dim=feature_dim,
            hidden_dim=encoder_dim,
            dropout=dropout
        )
        
        # Инициализация декодера
        self.decoder = Decoder(
            vocab_size=vocab_size,
            embed_dim=embed_dim,
            encoder_dim=encoder_dim * 2,  # Учитываем двунаправленность энкодера
            hidden_dim=decoder_dim,
            attention_dim=attention_dim,
            dropout=dropout
        )
        
    def forward(self, features, captions, lengths):
        """
        Прямой проход через модель
        
        Аргументы:
            features (torch.Tensor): Признаки видео [batch_size, seq_len, feature_dim]
            captions (torch.Tensor): Истинные подписи [batch_size, max_caption_length]
            lengths (list): Фактические длины подписей
            
        Возвращает:
            torch.Tensor: Предсказания модели [batch_size, max_caption_length, vocab_size]
        """
        # Кодируем видео
        encoder_outputs, _ = self.encoder(features)
        
        # Декодируем подписи
        outputs = self.decoder(encoder_outputs, captions, lengths)
        
        return outputs

In [ ]:
# 4. Функции обучения модели

def train_epoch(train_loader, model, criterion, optimizer,
                device, teacher_forcing_ratio):
    model.train()  # Переводим модель в режим обучения
    epoch_loss = 0.0  # Инициализация лосса для эпохи

    # Итерация по батчам данных
    for features, captions, lengths in train_loader:
        # Перенос данных на вычислительное устройство (GPU/CPU)
        features, captions = features.to(device), captions.to(device)
        optimizer.zero_grad()  # Обнуление градиентов

        # --- Прямой проход через энкодер ---
        encoder_outputs, _ = model.encoder(features)  # [B, H], где H - размер скрытого состояния
        batch_size = features.size(0)
        
        # Инициализация скрытых состояний декодера
        h, c = model.decoder.init_hidden_state(batch_size)
        
        # Начальный токен - <start> для всех примеров в батче
        inputs = captions[:, 0]  # [B]
        max_len = max(lengths)   # Максимальная длина подписи в текущем батче
        loss = 0.0  # Лосс для текущего батча

        # Цикл по временным шагам (пропускаем первый токен <start>)
        for t in range(1, max_len):
            # a) Эмбеддинг токенов
            emb = model.decoder.embedding(inputs)  # [B, embedding_dim]
            
            # b) Механизм внимания
            context, _ = model.decoder.attention(encoder_outputs, h)  # [B, hidden_size]
            
            # c) Объединение эмбеддинга и контекста
            lstm_input = torch.cat([emb, context], dim=1)  # [B, embedding_dim + hidden_size]
            
            # d) Шаг LSTM декодера
            h, c = model.decoder.lstm(lstm_input, (h, c))  # Обновление состояний
            
            # e) Преобразование скрытого состояния в логиты
            logits = model.decoder.fc(h)  # [B, vocab_size]
            
            # f) Расчет лосса между предсказанием и истинным токеном
            loss += criterion(logits, captions[:, t])  # Накопление лосса
            
            # g) Teacher Forcing: выбор следующего входного токена
            if random.random() < teacher_forcing_ratio:
                inputs = captions[:, t]  # Истинный предыдущий токен
            else:
                inputs = logits.argmax(dim=1)  # Предсказанный токен

        # Нормализация лосса по длине последовательности
        loss = loss / max_len
        # Обратное распространение ошибки
        loss.backward()
        # Обновление параметров модели
        optimizer.step()
        # Накопление лосса для эпохи
        epoch_loss += loss.item()

    # Возвращаем средний лосс на батч за эпоху
    return epoch_loss / len(train_loader)


def validate(model, dataloader, criterion, device):
    # Переводим модель в режим оценки (отключаем dropout и другие тренировочные механизмы)
    model.eval()
    total_loss = 0.0  # Инициализация общего лосса для эпохи валидации

    # Блок без вычисления градиентов для экономии памяти и ускорения вычислений
    with torch.no_grad():
        # Итерация по батчам данных
        for features, captions, lengths in dataloader:
            # Перенос данных на вычислительное устройство
            features, captions = features.to(device), captions.to(device)
            
            # Прямой проход через энкодер
            encoder_outputs, _ = model.encoder(features)  # [B, H]
            batch_size = features.size(0)
            
            # Инициализация скрытых состояний декодера
            h, c = model.decoder.init_hidden_state(batch_size)
            
            # Начальный токен <start> для всех примеров в батче
            inputs = captions[:, 0]  # [B]
            max_len = max(lengths)   # Максимальная длина в текущем батче
            loss = 0.0  # Лосс для текущего батча

            # Цикл генерации по временным шагам (без учительского форсинга)
            for t in range(1, max_len):
                # 1. Получение эмбеддинга токена
                emb = model.decoder.embedding(inputs)  # [B, embedding_dim]
                
                # 2. Вычисление контекста через механизм внимания
                context, _ = model.decoder.attention(encoder_outputs, h)  # [B, hidden_size]
                
                # 3. Объединение эмбеддинга и контекста
                lstm_input = torch.cat([emb, context], dim=1)  # [B, emb_dim + hidden_size]
                
                # 4. Обновление состояний LSTM
                h, c = model.decoder.lstm(lstm_input, (h, c))
                
                # 5. Преобразование в логиты словаря
                logits = model.decoder.fc(h)  # [B, vocab_size]
                
                # 6. Накопление лосса между предсказанием и истинным токеном
                loss += criterion(logits, captions[:, t])
                
                # 7. Всегда используем ground-truth токены (без teacher forcing)
                inputs = captions[:, t]  # Принудительная подача правильных токенов

            # Усреднение лосса по длине последовательности и добавление к общему лоссу
            total_loss += (loss / max_len).item()

    # Возвращаем средний лосс на батч за эпоху валидации
    return total_loss / len(dataloader)

In [ ]:
# 5. Основной цикл обучения модели

def train_model(model,
                train_loader,
                val_loader,
                criterion,
                optimizer,
                num_epochs,
                device,
                save_dir='checkpoints',
                save_every=5,
                scheduler=None,
                tf_start=1.0,
                tf_end=0.5):
    """
    Основная функция для обучения модели с валидацией и сохранением чекпоинтов.

    Параметры:
        model: nn.Module - модель для обучения
        train_loader, val_loader: DataLoader - загрузчики данных
        criterion: функция потерь
        optimizer: оптимизатор
        num_epochs: int - количество эпох обучения
        device: torch.device - устройство для вычислений (CPU/GPU)
        save_dir: str - директория для сохранения чекпоинтов
        save_every: int - периодичность сохранения чекпоинтов (в эпохах)
        scheduler: объект - планировщик скорости обучения (ReduceLROnPlateau)
        tf_start, tf_end: float - начальное и конечное значение коэффициента Teacher Forcing
    """
    # Создаем директорию для сохранения чекпоинтов
    os.makedirs(save_dir, exist_ok=True)
    
    # Инициализируем историю обучения
    history = {'train_loss': [], 'val_loss': []}

    # Основной цикл обучения по эпохам
    for epoch in range(1, num_epochs + 1):
        # 1) Расчет коэффициента Teacher Forcing для текущей эпохи
        # Линейное уменьшение от tf_start до tf_end в течение обучения
        tf_ratio = max(
            tf_end,  # Минимальное значение
            tf_start - (epoch - 1) * (tf_start - tf_end) / (num_epochs - 1)
        )

        # 2) Обучение на одной эпохе
        train_loss = train_epoch(
            train_loader,
            model,
            criterion,
            optimizer,
            device,
            tf_ratio
        )

        # 3) Валидация модели
        val_loss = validate(
            model,
            val_loader,
            criterion,
            device
        )

        # 4) Обновление скорости обучения (если используется планировщик)
        if scheduler is not None:
            scheduler.step(val_loss)  # Обычно для ReduceLROnPlateau

        # 5) Сохранение метрик и логирование
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        current_lr = optimizer.param_groups[0]['lr']  # Текущая скорость обучения
        
        # Вывод информации о прогрессе
        print(f"Эпоха {epoch}/{num_epochs}, "
              f"Ошибка (обучение): {train_loss:.4f}, "
              f"Ошибка (валидация): {val_loss:.4f}, "
              f"Скорость обучения: {current_lr:.6f}, "
              f"Teacher Forcing: {tf_ratio:.3f}")

        # 6) Сохранение промежуточных результатов
        if epoch % save_every == 0:
            # Формируем словарь с состоянием модели
            ckpt = {
                'epoch': epoch,
                'model_state': model.state_dict(),  # Веса модели
                'optim_state': optimizer.state_dict(),  # Состояние оптимизатора
                'train_loss': train_loss,
                'val_loss': val_loss,
                'lr': current_lr,
                'tf_ratio': tf_ratio
            }
            # Сохраняем чекпоинт
            path = os.path.join(save_dir, f"ckpt_epoch{epoch}.pt")
            torch.save(ckpt, path)
            print(f"Чекпоинт сохранен: {path}")

    # 7) Сохранение финальной модели после завершения обучения
    final_path = os.path.join(save_dir, "final_model.pt")
    torch.save({
        'model_state': model.state_dict(),
        'history': history  # Сохраняем историю обучения
    }, final_path)
    print(f"Финальная модель сохранена: {final_path}")

    return model, history

In [ ]:
# 6. Функции оценки качества модели

def predict_caption(model, features, vocab, beam_size=3, max_length=20):
    """
    Генерация подписи с использованием обученной модели и сбор весов внимания

    Параметры:
        model: Обученная модель (ожидается архитектура encoder-decoder с вниманием)
        features: Тензор признаков от энкодера [1, num_frames, feat_dim]
        vocab: Словарь для преобразования слов<->индексы
        beam_size: Параметр для beam search (не используется в текущей реализации)
        max_length: Максимальная длина генерируемой подписи

    Возвращает:
        caption_str: Строка сгенерированной подписи через пробел
        attention_weights: Список тензоров с весами внимания для каждого шага
    """
    # Переводим модель в режим инференса
    model.eval()
    
    # Инициализируем список для сбора весов внимания
    attention_weights = []
    
    # Отключаем вычисление градиентов для ускорения и экономии памяти
    with torch.no_grad():
        # Прямой проход через энкодер
        enc_out, _ = model.encoder(features)  # enc_out: [1, num_frames, feat_dim]
        
        # Инициализируем скрытые состояния декодера
        h, c = model.decoder.init_hidden_state(features.size(0))  # batch_size=1
        
        # Начальный токен <start> как вход для первого шага
        inputs = torch.LongTensor([vocab.word2idx['<start>']]).to(features.device)
        
        generated_idxs = []  # Список для сгенерированных индексов слов

        # Цикл генерации по временным шагам
        for _ in range(max_length):
            # 1. Получаем эмбеддинг текущего слова
            emb = model.decoder.embedding(inputs)  # [1, embedding_dim]
            
            # 2. Вычисляем контекст и веса внимания
            context, alpha = model.decoder.attention(enc_out, h)
            # alpha: [1, num_frames] - веса внимания для каждого кадра/региона
            
            # Сохраняем веса внимания (убираем batch-размер)
            attention_weights.append(alpha.squeeze(0).cpu())  # [num_frames]
            
            # 3. Объединяем эмбеддинг и контекст
            lstm_input = torch.cat([emb, context], dim=1)  # [1, emb_dim + feat_dim]
            
            # 4. Обновляем состояние LSTM
            h, c = model.decoder.lstm(lstm_input, (h, c))
            
            # 5. Получаем логиты для следующего слова
            logits = model.decoder.fc(h)  # [1, vocab_size]
            
            # 6. Выбираем наиболее вероятный токен (жадный алгоритм)
            inputs = logits.argmax(1)  # [1]
            
            # Проверяем условие завершения генерации
            if inputs.item() == vocab.word2idx['<end>']:
                break
                
            generated_idxs.append(inputs.item())

    # Преобразование индексов в слова через словарь
    words = [vocab.idx2word[idx] for idx in generated_idxs]
    
    # Возвращаем результат и историю внимания
    return ' '.join(words), attention_weights


def caption_collate_fn(batch, vocab):
    """Пользовательская функция для обработки батчей с подписями разной длины"""
    # Сортируем батч по длине подписей (по убыванию)
    batch.sort(key=lambda x: len(x[1]), reverse=True)
    features, captions = zip(*batch)
    
    # Объединяем признаки
    features = torch.stack(features, 0)
    
    # Получаем длины подписей
    lengths = torch.LongTensor([len(cap) for cap in captions])
    
    # Определяем максимальную длину для паддинга
    max_length = max(lengths)
    
    # Создаем тензор с заполненными подписями
    padded_captions = torch.zeros(len(captions), max_length, dtype=torch.long)
    for i, cap in enumerate(captions):
        end = lengths[i]
        padded_captions[i, :end] = cap[:end]
        # Добавляем токен <end> в конец последовательности
        if end < max_length:
            padded_captions[i, end] = vocab.word2idx['<end>']
    
    return features, padded_captions, lengths


def visualize_attention(video_frames, caption, attention_weights, grid_shape=(5, 8)):
    """
    Визуализирует веса внимания модели в виде тепловых карт для каждого слова подписи

    Параметры:
        video_frames (list): Список кадров видео (не используется в текущей реализации)
        caption (str): Сгенерированная текстовая подпись
        attention_weights (list): Список тензоров с весами внимания для каждого слова
        grid_shape (tuple): Размерность сетки для визуализации (строки, столбцы)
    """
    # Разделяем подпись на отдельные слова
    words = caption.split()
    
    # Проверка согласованности данных
    if len(words) != len(attention_weights):
        print(f"Предупреждение: Количество слов ({len(words)}) не совпадает с количеством карт внимания ({len(attention_weights)})")
        return

    # Создаем сетку графиков (вертикальное расположение)
    rows = len(words)
    fig, axes = plt.subplots(rows, 1, figsize=(12, 3*rows))
    
    # Обрабатываем случай с одной подписью (исправление размерности осей)
    if rows == 1:
        axes = [axes]

    # Генерация тепловых карт для каждого слова
    for i, (word, weights) in enumerate(zip(words, attention_weights)):
        ax = axes[i]
        try:
            # 1. Преобразование весов в 2D сетку
            attention_grid = weights.numpy().reshape(grid_shape)
            
            # 2. Отображение тепловой карты
            im = ax.imshow(attention_grid, 
                          cmap='viridis', 
                          interpolation='nearest')
            
            # 3. Добавление цветовой шкалы
            plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
            
            # 4. Настройка оформления
            ax.set_title(f"Слово: '{word}'", fontsize=12, pad=10)
            ax.axis('off')
            
        except Exception as e:
            print(f"Ошибка визуализации для слова '{word}': {str(e)}")
            ax.axis('off')  # Скрываем поврежденные оси

    # Оптимизация расположения элементов
    plt.tight_layout(pad=2.0)
    plt.show()


def evaluate_metrics(model, dataloader, vocab, device):
    """Оценка качества модели по BLEU-4
    
    Аргументы:
        model: Обученная модель
        dataloader: Загрузчик данных для оценки
        vocab: Словарь
        device: Устройство вычислений
        
    Возвращает:
        dict: Словарь с метриками качества
    """
    model.eval()
    references = []
    hypotheses = []
    
    with torch.no_grad():
        for features, captions, lengths in tqdm(dataloader, desc='Оценка'):
            # Генерация подписи
            pred_caption, _ = predict_caption(model, features.to(device), vocab)
            hypotheses.append(pred_caption.split())
            
            # Обработка эталонной подписи
            ref = []
            for idx in captions[0].cpu().numpy():
                if idx == vocab.word2idx['<end>']:
                    break
                if idx not in [vocab.word2idx['<pad>'], vocab.word2idx['<start>']]:
                    ref.append(vocab.idx2word[idx])
            references.append([ref])
        
    # Вычисление метрик
    bleu4 = nltk.translate.bleu_score.corpus_bleu(
        references, hypotheses, weights=(0.25, 0.25, 0.25, 0.25))
    
    return {'bleu4': bleu4}


def find_optimal_batch_size(model, sample_data, criterion, max_batch_size=16):
    """Автоматический подбор оптимального размера батча
    
    Аргументы:
        model: Модель для тестирования
        sample_data: Пример данных (features, captions)
        criterion: Функция потерь для оценки результата
        max_batch_size: Максимальный размер батча для проверки
        
    Возвращает:
        int: Оптимальный размер батча
    """
    model.train()
    optimal_size = 1
    features, captions, lengths = sample_data
    
    for bs in range(1, max_batch_size + 1, 2):
        try:
            # Тестовый прогон
            batch_features = features.repeat(bs, 1, 1).to(device)
            batch_captions = captions.repeat(bs, 1).to(device)
            batch_lengths = lengths.repeat(bs) # Создаем batch_lengths

            # Передаем все три параметра в модель
            outputs = model(batch_features, batch_captions, batch_lengths)
            loss = criterion(outputs.view(-1, outputs.size(-1)), batch_captions.view(-1))
            loss.backward()
            
            optimal_size = bs
            print(f"Батч {bs} успешно обработан")
            
            # Очистка памяти
            del batch_features, batch_captions, batch_lengths, outputs, loss
            torch.cuda.empty_cache()
            
        except RuntimeError as e:
            if "памяти" in str(e).lower():
                print(f"Достигнут лимит памяти при батче {bs}")
                break
                
    print(f"Оптимальный размер батча: {optimal_size}")
    return optimal_size

In [ ]:
# 7. Визуализация

def visualize_predictions(model, dataloader, vocab, num_samples=5, device='cuda'):
    """
    Визуализация предсказаний модели и тепловых карт внимания.

    Args:
        model: обученная модель
        dataloader: загрузчик тестовых данных
        vocab: объект Vocabulary
        num_samples: сколько примеров показать
        device: 'cuda' или 'cpu'
    """
    model.eval()
    samples = []
    count = 0

    # Собираем несколько примеров
    with torch.no_grad():
        for features, captions, lengths in dataloader:
            if count >= num_samples:
                break
            features = features.to(device)
            pred_caption, attn_weights = predict_caption(model, features, vocab)

            # decode ground truth
            gt_idxs = captions[0].cpu().tolist()
            gt_words = []
            for idx in gt_idxs:
                if idx == vocab.word2idx['<end>']:
                    break
                if idx not in {vocab.word2idx['<pad>'],
                               vocab.word2idx['<start>']}:
                    gt_words.append(vocab.idx2word[idx])
            gt_caption = ' '.join(gt_words)

            samples.append((gt_caption, pred_caption, attn_weights))
            count += 1

    # Рисуем для каждого примера текст + heatmap
    fig, axes = plt.subplots(num_samples, 2, figsize=(12, 4 * num_samples))
    if num_samples == 1:
        axes = np.expand_dims(axes, 0)

    for i, (gt, pred, attn) in enumerate(samples):
        ax_text, ax_hm = axes[i]

        # 1) Текстовая часть
        ax_text.axis('off')
        ax_text.text(0, 0.6, f"GT:  {gt}", fontsize=12)
        ax_text.text(0, 0.3, f"PR:  {pred}", fontsize=12, color='blue')

        # 2) Тепловая карта внимания
        # Превращаем list[tensor] → numpy matrix
        attn_numpy = np.vstack([
            w.detach().cpu().reshape(1, -1) for w in attn
        ])
        im = ax_hm.imshow(attn_numpy,
                          aspect='auto',
                          cmap='viridis')
        ax_hm.set_title("Attention Heatmap")
        ax_hm.set_ylabel("Шаг слова")
        ax_hm.set_xlabel("Позиция кадра")
        # подписи по осям Y - слова предсказания
        words = pred.split()
        ax_hm.set_yticks(range(len(words)))
        ax_hm.set_yticklabels(words, fontsize=10)
        plt.colorbar(im, ax=ax_hm, fraction=0.046, pad=0.04)

    plt.tight_layout()
    plt.show()